# Tensorflow MNIST-LeNet Patch Demo

This notebook demonstrates the adversarial patch attack applied on the LeNet model, as well as preprocessing and adversarial training defenses.

The following two sections cover experiment setup and is similar across all demos.

## Setup: Experiment Name and MNIST Dataset

Here we will import the necessary Python modules and ensure the proper environment variables are set so that all the code blocks will work as expected.

**Important: Users will need to verify or update the following parameters:**

- Ensure that the `USERNAME` parameter is set to your own name.
- Ensure that the `DATASET_DIR` parameter is set to the location of the MNIST dataset directory. Currently set to `/nfs/data/Mnist` as the default location.
- (Optional) Set the `EXPERIMENT_NAME` parameter to your own preferred experiment name.

Other parameters can be modified to alter the RESTful API and MLFlow tracking addresses. 

In [1]:
# Import packages from the Python standard library
import os
import pprint
import time
import warnings
from pathlib import Path
from typing import Tuple

# Filter out warning messages
warnings.filterwarnings("ignore")

# Please enter custom username here.
USERNAME = "howard"

# Ensure that the dataset location is properly set here.
DATASET_DIR = "/nfs/data/Mnist"

# Experiment name (note the username_ prefix convention)
EXPERIMENT_NAME = f"{USERNAME}_mnist_adversarial_patches"

# Address for connecting the docker container to exposed ports on the host device
HOST_DOCKER_INTERNAL = "host.docker.internal"
# HOST_DOCKER_INTERNAL = "172.17.0.1"

# Testbed API ports
RESTAPI_PORT = "30080"
MLFLOW_TRACKING_PORT = "35000"

# Default address for accessing the RESTful API service
RESTAPI_ADDRESS = (
    f"http://{HOST_DOCKER_INTERNAL}:{RESTAPI_PORT}"
    if os.getenv("IS_JUPYTER_SERVICE")
    else f"http://localhost:{RESTAPI_PORT}"
)

# Override the AI_RESTAPI_URI variable, used to connect to RESTful API service
os.environ["AI_RESTAPI_URI"] = RESTAPI_ADDRESS

# Default address for accessing the MLFlow Tracking server
MLFLOW_TRACKING_URI = (
    f"http://{HOST_DOCKER_INTERNAL}:{MLFLOW_TRACKING_PORT}"
    if os.getenv("IS_JUPYTER_SERVICE")
    else f"http://localhost:{MLFLOW_TRACKING_PORT}"
)

# Path to custom task plugins archives
CUSTOM_PLUGINS_EVALUATION_TAR_GZ = Path("custom-plugins-evaluation.tar.gz")

# Override the MLFLOW_TRACKING_URI variable, used to connect to MLFlow Tracking service
os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI

# Base API address
RESTAPI_API_BASE = f"{RESTAPI_ADDRESS}/api"

# Path to workflows archive
WORKFLOWS_TAR_GZ = Path("workflows.tar.gz")

# Import third-party Python packages
import numpy as np
import requests
from mlflow.tracking import MlflowClient

# Import utils.py file
import utils

# Create random number generator
rng = np.random.default_rng(54399264723942495723666216079516778448)

## Dataset

The training and testing images in this directory are saved as PNG files and are organized into the following folder structure,

    Mnist
    ├── testing
    │   ├── 0
    │   ├── 1
    │   ├── 2
    │   ├── 3
    │   ├── 4
    │   ├── 5
    │   ├── 6
    │   ├── 7
    │   ├── 8
    │   └── 9
    └── training
        ├── 0
        ├── 1
        ├── 2
        ├── 3
        ├── 4
        ├── 5
        ├── 6
        ├── 7
        ├── 8
        └── 9

The subfolders under `Mnist/training/` and `Mnist/testing/` are the classification labels for the images in the dataset.
This folder structure is a standardized way to encode the label information and many libraries can make use of it, including the Tensorflow library that we are using for this particular demo.

## Submit and run jobs

The entrypoints that we will be running in this example are implemented in the Python source files under `src/` and the `MLproject` file.
To run these entrypoints within the testbed architecture, we need to package those files up into an archive and submit it to the Testbed RESTful API to create a new job.
For convenience, the `Makefile` provides a rule for creating the archive file for this example, just run `make workflows`,

In [2]:
%%bash

# Create the workflows.tar.gz file
make workflows

make: Nothing to be done for 'workflows'.


/home/hhuang/.conda/envs/tensorflow-mnist-classifier/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


To connect with the endpoint, we will use a client class defined in the `utils.py` file that is able to connect with the Testbed RESTful API using the HTTP protocol.
We connect using the client below, which uses the environment variable `AI_RESTAPI_URI` to figure out how to connect to the Testbed RESTful API,

In [3]:
restapi_client = utils.SecuringAIClient()

/home/hhuang/.conda/envs/tensorflow-mnist-classifier/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


We need to register an experiment under which to collect our job runs.
The code below checks if the relevant experiment exists.
If it does, then it just returns info about the experiment, if it doesn't, it then registers the new experiment.

In [4]:
response_experiment = restapi_client.get_experiment_by_name(name=EXPERIMENT_NAME)

if response_experiment is None or "Not Found" in response_experiment.get("message", []):
    response_experiment = restapi_client.register_experiment(name=EXPERIMENT_NAME)

response_experiment

{'createdOn': '2020-11-05T08:45:14.946823',
 'experimentId': 10,
 'lastModified': '2020-11-05T08:45:14.946823',
 'name': 'howard_mnist_adversarial_patches'}

We should also check which queues are available for running our jobs to make sure that the resources that we need are available.
The code below queries the Testbed API and returns a list of active queues.

In [5]:
restapi_client.list_queues()

[{'createdOn': '2020-11-20T17:46:06.756687',
  'queueId': 1,
  'lastModified': '2020-11-20T17:46:06.756687',
  'name': 'tensorflow_cpu'},
 {'createdOn': '2020-11-20T18:00:40.876888',
  'queueId': 2,
  'lastModified': '2020-11-20T18:00:40.876888',
  'name': 'tensorflow_gpu'},
 {'createdOn': '2020-11-20T19:52:36.079781',
  'queueId': 5,
  'lastModified': '2020-11-20T19:52:36.079781',
  'name': 'pytorch_cpu'},
 {'createdOn': '2020-11-20T19:52:43.348460',
  'queueId': 7,
  'lastModified': '2020-11-20T19:52:43.348460',
  'name': 'pytorch_gpu'}]

This example also makes use of the `custom_patch_plugins` custom task plugin package stored locally under the `task-plugins/securingai_custom/custom_patch_plugins` directory.
To register these custom task plugins, we first need to package them up into an archive.
For convenience, the `Makefile` provides a rule for creating the custom task plugins archive file, just run `make custom-plugins`,

In [6]:
%%bash

# Create the workflows.tar.gz file
make custom-plugins

make: Nothing to be done for 'custom-plugins'.


Now that the custom task plugin package is packaged into an archive file, next we register it by uploading the file to the REST API.
Note that we need to provide the name to use for custom task plugin package, this name must be unique under the custom task plugins namespace.
For a full list of the custom task plugins, use `restapi_client.restapi_client.list_custom_task_plugins()`.

In [7]:
restapi_client.delete_custom_task_plugin(name="custom_patch_plugins")
response_custom_plugins = restapi_client.get_custom_task_plugin(name="custom_patch_plugins")

if response_custom_plugins is None or "Not Found" in response_custom_plugins.get("message", []):
    response_custom_plugins = restapi_client.upload_custom_plugin_package(
        custom_plugin_name="custom_patch_plugins",
        custom_plugin_file=CUSTOM_PLUGINS_EVALUATION_TAR_GZ,
    )

response_custom_plugins

/home/hhuang/.conda/envs/tensorflow-mnist-classifier/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'taskPluginName': 'custom_patch_plugins',
 'collection': 'securingai_custom',
 'modules': ['data_tensorflow.py',
  'tensorflow.py',
  'attacks_patch.py',
  'registry_art.py',
  '__init__.py',
  'estimators_keras_classifiers.py',
  'import_keras.py',
  'defenses_image_preprocessing.py']}

If at any point you need to update one or more files within the `evaluation` plugin package, you will need to unregister/delete the custom task plugin first using the REST API.
This can be done as follows,

```python
# Delete the 'evaluation' custom task plugin package
restapi_client.delete_custom_task_plugin(name="custom_patch_plugins")
```

## Adversarial Patches: Baseline MNIST Training

Now, we will train our baseline LeNet-5 model on the MNIST dataset. 
We will be submitting our jobs to the `"tensorflow_gpu"` queue.
Once the experiment is finished, we will examine the accuracy results of our model.

In [8]:
# Create and submit training job.
response_le_net_train = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="train",
    entry_point_kwargs=" ".join([
        "-P batch_size=256",
        f"-P register_model_name={EXPERIMENT_NAME}_le_net",
        "-P model_architecture=le_net",
        "-P epochs=30",
        f"-P data_dir_training={DATASET_DIR}/training",
        f"-P data_dir_testing={DATASET_DIR}/testing",
    ]),
    queue="tensorflow_gpu",
    timeout="1h",
)

print("Training job for LeNet-5 neural network submitted")
print("")
pprint.pprint(response_le_net_train)

Training job for LeNet-5 neural network submitted

{'createdOn': '2021-09-23T07:10:34.535550',
 'dependsOn': None,
 'entryPoint': 'train',
 'entryPointKwargs': '-P batch_size=256 -P '
                     'register_model_name=howard_mnist_adversarial_patches_le_net '
                     '-P model_architecture=le_net -P epochs=30 -P '
                     'data_dir_training=/nfs/data/Mnist/training -P '
                     'data_dir_testing=/nfs/data/Mnist/testing',
 'experimentId': 10,
 'jobId': 'a52bb183-924f-491d-919e-7958e5371870',
 'lastModified': '2021-09-23T07:10:34.535550',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '1h',
 'workflowUri': 's3://workflow/d77c967c88f24d6cbc72f459183907d6/workflows.tar.gz'}


The following helper functions will recheck the job responses until the job is completed or a run ID is available. 
The run ID is needed to link dependencies between jobs.

In [9]:
def mlflow_run_id_is_not_known(job_response):
    return job_response["mlflowRunId"] is None and job_response["status"] not in [
        "failed",
        "finished",
    ]


def get_run_id(job_response):
    while mlflow_run_id_is_not_known(job_response):
        time.sleep(1)
        job_response = restapi_client.get_job_by_id(job_response["jobId"])
        
    return job_response


def wait_until_finished(job_response):
    # First make sure job has started.
    job_response = get_run_id(job_response)
    
    # Next re-check job until it has stopped running.
    while (job_response["status"] not in ["failed", "finished"]):
        time.sleep(1)
        job_response = restapi_client.get_job_by_id(job_response["jobId"])
    
    return job_response

Now wait for the job to complete before proceeding to next steps.

In [10]:
response_le_net_train = wait_until_finished(response_le_net_train)
print("Training job for LeNet-5 neural network")
pprint.pprint(response_le_net_train)

Training job for LeNet-5 neural network
{'createdOn': '2021-09-23T07:10:34.535550',
 'dependsOn': None,
 'entryPoint': 'train',
 'entryPointKwargs': '-P batch_size=256 -P '
                     'register_model_name=howard_mnist_adversarial_patches_le_net '
                     '-P model_architecture=le_net -P epochs=30 -P '
                     'data_dir_training=/nfs/data/Mnist/training -P '
                     'data_dir_testing=/nfs/data/Mnist/testing',
 'experimentId': 10,
 'jobId': 'a52bb183-924f-491d-919e-7958e5371870',
 'lastModified': '2021-09-23T07:26:43.454882',
 'mlflowRunId': '846c061b5d444b3a909f41fadf3fef9f',
 'queueId': 2,
 'status': 'finished',
 'timeout': '1h',
 'workflowUri': 's3://workflow/d77c967c88f24d6cbc72f459183907d6/workflows.tar.gz'}


### Checking baseline MNIST job accuracy

Once the job has finished running we can view the results either through the MLflow URI or by accessing the job via MLflow client.
Here we will show the baseline accuracy results from the previous training job.
Please see [Querying the MLFlow Tracking Service](#Querying-the-MLFlow-Tracking-Service) section for more details.

In [11]:
# Helper function for viewing MLflow results.
def get_mlflow_results(job_response):
    mlflow_client = MlflowClient()
    job_response = wait_until_finished(job_response)
    if(job_response['status']=="failed"):
        return {}
    
    run = mlflow_client.get_run(job_response["mlflowRunId"])  
    while(len(run.data.metrics) == 0):
        time.sleep(1)
        run = mlflow_client.get_run(job_response["mlflowRunId"])
        
    return run


results = get_mlflow_results(response_le_net_train)
pprint.pprint(results.data.metrics)

{'accuracy': 0.9838763475418091,
 'auc': 0.9993220567703247,
 'loss': 0.05461121819268966,
 'precision': 0.9859369993209839,
 'recall': 0.9814390540122986,
 'restored_epoch': 4.0,
 'stopped_epoch': 9.0,
 'training_time_in_minutes': 6.7917063,
 'val_accuracy': 0.9869956374168396,
 'val_auc': 0.9990764856338501,
 'val_loss': 0.043413427410686906,
 'val_precision': 0.9886373281478882,
 'val_recall': 0.9864121079444885}


With the baseline training settings, it appears that the LeNet model has been properly trained on MNIST dataset.

### Deploying and Testing Adversarial Patches

Now we will create and apply the adversarial patches over our test set and evaluate the performance of the baseline model on the adversarial patches.
We will also apply the patches over the training set for the adversarial training defense evaluation.

#### Patch Generation

The following job will generate the adversarial patches. 
Feel free to adjust the input parameters to see how they impact the effectiveness of the patch attack.

In [12]:
# Create Patches
response_le_net_patches = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="gen_patch",
    entry_point_kwargs=" ".join(
        [
            f"-P model_name={EXPERIMENT_NAME}_le_net",
            f"-P model_version=none",
            f"-P data_dir={DATASET_DIR}/training",
            "-P num_patch_gen_samples=40",
            "-P num_patch=3",
            "-P patch_target=5",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_le_net_train["jobId"],
)


print("Patch attack (LeNet-5 architecture) job submitted")
print("")
pprint.pprint(response_le_net_patches)
print("")

# Wait for Patch attack to finish.
response_le_net_patches = wait_until_finished(response_le_net_patches)

Patch attack (LeNet-5 architecture) job submitted

{'createdOn': '2021-09-23T07:26:43.775300',
 'dependsOn': 'a52bb183-924f-491d-919e-7958e5371870',
 'entryPoint': 'gen_patch',
 'entryPointKwargs': '-P model_name=howard_mnist_adversarial_patches_le_net -P '
                     'model_version=none -P data_dir=/nfs/data/Mnist/training '
                     '-P num_patch_gen_samples=40 -P num_patch=3 -P '
                     'patch_target=5',
 'experimentId': 10,
 'jobId': '661f7ac9-251a-47dd-83ea-a54bd2cc196a',
 'lastModified': '2021-09-23T07:26:43.775300',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/ed368e4a40cf4a3e868a86c17e52a37f/workflows.tar.gz'}



#### Patch Deployment

The following jobs will deploy the patches over the training and test sets.

In [13]:
# Deploy Patch attack on training set.

response_deploy_le_net_patches_mnist_training = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="deploy_patch",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_le_net_patches['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_le_net",
            f"-P model_version=none",
            f"-P data_dir={DATASET_DIR}/training",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_le_net_patches["jobId"],
)

print("Patch deployment (LeNet-5 architecture) job submitted")
print("")
pprint.pprint(response_deploy_le_net_patches_mnist_training)
print("")

# Get the run ID of the test set.
response_deploy_le_net_patches_mnist_training = get_run_id(response_deploy_le_net_patches_mnist_training)

Patch deployment (LeNet-5 architecture) job submitted

{'createdOn': '2021-09-23T07:31:38.533225',
 'dependsOn': '661f7ac9-251a-47dd-83ea-a54bd2cc196a',
 'entryPoint': 'deploy_patch',
 'entryPointKwargs': '-P run_id=1045581a0d8c42cea154406baf113914 -P '
                     'model_name=howard_mnist_adversarial_patches_le_net -P '
                     'model_version=none -P data_dir=/nfs/data/Mnist/training',
 'experimentId': 10,
 'jobId': 'b7963146-e4b7-42a0-a37d-631336ed6dd8',
 'lastModified': '2021-09-23T07:31:38.533225',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/7cb2c28cd1fc4d8e9c9d91522e524789/workflows.tar.gz'}



In [14]:
# Deploy Patch attack on test set.
response_deploy_le_net_patches_mnist_testing = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="deploy_patch",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_le_net_patches['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_le_net",
            f"-P model_version=none",
            f"-P data_dir={DATASET_DIR}/testing",
            "-P patch_deployment_method=corrupt"
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_le_net_patches["jobId"],
)

print("Patch deployment (LeNet-5 architecture) job submitted")
print("")
pprint.pprint(response_deploy_le_net_patches_mnist_testing)
print("")

# Get the run ID of the training set.
response_deploy_le_net_patches_mnist_testing = get_run_id(response_deploy_le_net_patches_mnist_testing)

Patch deployment (LeNet-5 architecture) job submitted

{'createdOn': '2021-09-23T07:32:01.933182',
 'dependsOn': '661f7ac9-251a-47dd-83ea-a54bd2cc196a',
 'entryPoint': 'deploy_patch',
 'entryPointKwargs': '-P run_id=1045581a0d8c42cea154406baf113914 -P '
                     'model_name=howard_mnist_adversarial_patches_le_net -P '
                     'model_version=none -P data_dir=/nfs/data/Mnist/testing '
                     '-P patch_deployment_method=corrupt',
 'experimentId': 10,
 'jobId': '297a740b-3289-4a5f-868a-4fcc95f62ff6',
 'lastModified': '2021-09-23T07:32:01.933182',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/6ca3db6d44d949ff8179b56c32eb2a4b/workflows.tar.gz'}



### Patch Attack Evaluation: Baseline MNIST Model

Now we will run an inference step to check the patch-attacked dataset with our MNIST-trained model.

In [15]:
# Check patched dataset results   
response_le_net_infer_le_net_patch = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_deploy_le_net_patches_mnist_testing['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_le_net",
            f"-P model_version=none",
            "-P batch_size=512",
            "-P adv_tar_name=adversarial_patch_dataset.tar.gz",
            "-P adv_data_dir=adv_patch_dataset",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_deploy_le_net_patches_mnist_testing["jobId"],
)
        
print("Patch evaluation (LeNet-5 architecture) job submitted")
print("")
pprint.pprint(response_deploy_le_net_patches_mnist_testing)
print("")

Patch evaluation (LeNet-5 architecture) job submitted

{'createdOn': '2021-09-23T07:32:01.933182',
 'dependsOn': '661f7ac9-251a-47dd-83ea-a54bd2cc196a',
 'entryPoint': 'deploy_patch',
 'entryPointKwargs': '-P run_id=1045581a0d8c42cea154406baf113914 -P '
                     'model_name=howard_mnist_adversarial_patches_le_net -P '
                     'model_version=none -P data_dir=/nfs/data/Mnist/testing '
                     '-P patch_deployment_method=corrupt',
 'experimentId': 10,
 'jobId': '297a740b-3289-4a5f-868a-4fcc95f62ff6',
 'lastModified': '2021-09-23T07:37:00.468727',
 'mlflowRunId': '26b2f57a935b418ba240508a6aecba0b',
 'queueId': 2,
 'status': 'started',
 'timeout': '24h',
 'workflowUri': 's3://workflow/6ca3db6d44d949ff8179b56c32eb2a4b/workflows.tar.gz'}



In [16]:
# Wait for the job to finish
response_le_net_infer_le_net_patch = wait_until_finished(response_le_net_infer_le_net_patch)

# Check on the patch evaluation results
results = get_mlflow_results(response_le_net_infer_le_net_patch)
print("Baseline model results on adversarially patched dataset: ")
pprint.pprint(results.data.metrics)

Baseline model results on adversarially patched dataset: 
{'accuracy': 0.3639823794364929,
 'auc': 0.7619609832763672,
 'loss': 1.9739676356315612,
 'precision': 0.6918551921844482,
 'recall': 0.30629006028175354}



We can see that the adversarial patch attack causes a noticeable decrease in the model's accuracy scores.

We will now test various defenses against the patch attacked images.

## Defenses: Image Preprocessing and Adversarial Training

The next part of the adversarial patch demo focuses on investigating effective defenses against the attack.

### Preprocessing Defenses: Spatial Smoothing, JPEG Compression, Gaussian Augmentation

Here we will investigate three preprocessing defenses that can be applied over the images before inference.

#### Spatial Smoothing Defense:

Here, we can adjust the `spatial_smoothing_window_size` parameter to increase or decrease the sliding window of the smoothing defense.
Larger values will create more noticeable distortions but can also help mask any adversarial perturbations.

In [17]:
response_le_net_spatial_smoothing_test_set = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="spatial_smoothing",
    entry_point_kwargs=" ".join(
        [
            f"-P data_dir={DATASET_DIR}/training",
            "-P batch_size=20",
            "-P load_dataset_from_mlruns=true",
            "-P spatial_smoothing_window_size=2",
            f"-P dataset_run_id={response_deploy_le_net_patches_mnist_testing['mlflowRunId']}",
            "-P dataset_tar_name=adversarial_patch_dataset.tar.gz",
            "-P dataset_name=adv_patch_dataset",
            
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_deploy_le_net_patches_mnist_testing["jobId"],
)

print("Spatial smoothing defense job submitted")
print("")
pprint.pprint(response_le_net_spatial_smoothing_test_set)
print("")

response_le_net_spatial_smoothing_test_set = get_run_id(response_le_net_spatial_smoothing_test_set)

Spatial smoothing defense job submitted

{'createdOn': '2021-09-23T07:38:05.975857',
 'dependsOn': '297a740b-3289-4a5f-868a-4fcc95f62ff6',
 'entryPoint': 'spatial_smoothing',
 'entryPointKwargs': '-P data_dir=/nfs/data/Mnist/training -P batch_size=20 -P '
                     'load_dataset_from_mlruns=true -P '
                     'spatial_smoothing_window_size=2 -P '
                     'dataset_run_id=26b2f57a935b418ba240508a6aecba0b -P '
                     'dataset_tar_name=adversarial_patch_dataset.tar.gz -P '
                     'dataset_name=adv_patch_dataset',
 'experimentId': 10,
 'jobId': 'fb3b3e92-d541-48ad-8ee9-4f1ec8c6012d',
 'lastModified': '2021-09-23T07:38:05.975857',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/1de0bb9bbc9a41449859544ba6199718/workflows.tar.gz'}



In [18]:
# Wait for defense to complete, then check baseline model on defended test set with adversarial patches.

response_evaluate_spatial_smoothing_images = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_le_net_spatial_smoothing_test_set['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_le_net",
            f"-P model_version=none",
            "-P batch_size=20",
            "-P adv_tar_name=spatial_smoothing_dataset.tar.gz",
            "-P adv_data_dir=adv_testing",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_le_net_spatial_smoothing_test_set["jobId"],
)

print("Inference job submitted")
print("")
pprint.pprint(response_evaluate_spatial_smoothing_images)
print("")

response_evaluate_spatial_smoothing_images = wait_until_finished(response_evaluate_spatial_smoothing_images)
results = get_mlflow_results(response_evaluate_spatial_smoothing_images)
print("Adversarial Patches with Spatial Smoothing Results: \n")
pprint.pprint(results.data.metrics)

Inference job submitted

{'createdOn': '2021-09-23T07:38:13.152706',
 'dependsOn': 'fb3b3e92-d541-48ad-8ee9-4f1ec8c6012d',
 'entryPoint': 'infer',
 'entryPointKwargs': '-P run_id=0af78ff1900d43a19fb9122c62a97422 -P '
                     'model_name=howard_mnist_adversarial_patches_le_net -P '
                     'model_version=none -P batch_size=20 -P '
                     'adv_tar_name=spatial_smoothing_dataset.tar.gz -P '
                     'adv_data_dir=adv_testing',
 'experimentId': 10,
 'jobId': '1037f7af-072f-4e64-a3a5-3668007dd957',
 'lastModified': '2021-09-23T07:38:13.152706',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/3dc417eb1ace476e93e10811433ab650/workflows.tar.gz'}

Adversarial Patches with Spatial Smoothing Results: 

{'accuracy': 0.36833667755126953,
 'auc': 0.7662726640701294,
 'loss': 1.8915234726512122,
 'precision': 0.7604895234107971,
 'recall': 0.30511021614074707}


It appears that spatial smoothing does not protect well against adversarial patches.

#### JPEG Compression Defense:

We can adjust the image compression quality by modifying the `jpeg_compression_quality` field. 
Enter any value between 1 (worst) to 95 (best).

In [19]:
response_le_net_jpeg_compression_test_set = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="jpeg_compression",
    entry_point_kwargs=" ".join(
        [
            f"-P data_dir=/{DATASET_DIR}/training",
            "-P batch_size=20",
            "-P load_dataset_from_mlruns=true",
            f"-P dataset_run_id={response_deploy_le_net_patches_mnist_testing['mlflowRunId']}",
            "-P jpeg_compression_quality=30",
            "-P dataset_tar_name=adversarial_patch_dataset.tar.gz",
            "-P dataset_name=adv_patch_dataset",
            
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_deploy_le_net_patches_mnist_testing["jobId"],
)

print("JPEG compression defense job submitted")
print("")
pprint.pprint(response_le_net_jpeg_compression_test_set)
print("")

response_le_net_jpeg_compression_test_set = get_run_id(response_le_net_jpeg_compression_test_set)

JPEG compression defense job submitted

{'createdOn': '2021-09-23T07:39:02.041165',
 'dependsOn': '297a740b-3289-4a5f-868a-4fcc95f62ff6',
 'entryPoint': 'jpeg_compression',
 'entryPointKwargs': '-P data_dir=//nfs/data/Mnist/training -P batch_size=20 '
                     '-P load_dataset_from_mlruns=true -P '
                     'dataset_run_id=26b2f57a935b418ba240508a6aecba0b -P '
                     'jpeg_compression_quality=30 -P '
                     'dataset_tar_name=adversarial_patch_dataset.tar.gz -P '
                     'dataset_name=adv_patch_dataset',
 'experimentId': 10,
 'jobId': '9f9b7911-b690-4c2a-b234-c9dfba4aec3f',
 'lastModified': '2021-09-23T07:39:02.041165',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/6de60d3bcb954e1f948f8eb509e991ed/workflows.tar.gz'}



In [20]:
# Wait for defense to complete, then check baseline model on defended test set with adversarial patches.

response_evaluate_jpeg_compression_images = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_le_net_jpeg_compression_test_set['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_le_net",
            f"-P model_version=none",
            "-P batch_size=20",
            "-P adv_tar_name=jpeg_compression_dataset.tar.gz",
            "-P adv_data_dir=adv_testing",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_le_net_jpeg_compression_test_set["jobId"],
)

print("Inference job submitted")
print("")
pprint.pprint(response_evaluate_jpeg_compression_images)
print("")

response_evaluate_jpeg_compression_images = wait_until_finished(response_evaluate_jpeg_compression_images)
results = get_mlflow_results(response_evaluate_jpeg_compression_images)
print("Adversarial Patches with JPEG Compression Results: \n")
pprint.pprint(results.data.metrics)


Inference job submitted

{'createdOn': '2021-09-23T07:39:09.221540',
 'dependsOn': '9f9b7911-b690-4c2a-b234-c9dfba4aec3f',
 'entryPoint': 'infer',
 'entryPointKwargs': '-P run_id=d0367faf495d4c109f10aaaef521399e -P '
                     'model_name=howard_mnist_adversarial_patches_le_net -P '
                     'model_version=none -P batch_size=20 -P '
                     'adv_tar_name=jpeg_compression_dataset.tar.gz -P '
                     'adv_data_dir=adv_testing',
 'experimentId': 10,
 'jobId': '36dcada7-653e-4090-ab8e-7c2d2802844c',
 'lastModified': '2021-09-23T07:39:09.221540',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/c5e6262baccb4c458ec12c9b0824c5e6/workflows.tar.gz'}

Adversarial Patches with JPEG Compression Results: 

{'accuracy': 0.3654308617115021,
 'auc': 0.7599896192550659,
 'loss': 2.003694204385868,
 'precision': 0.6728951334953308,
 'recall': 0.30671343207359314}


#### Gaussian Augmentation Defense:

We can adjust the amount of noise created by the defense by adjusting the `gaussian_augmentation_sigma` parameter. 
Please enter any positive value for sigma.

In [21]:
response_le_net_gaussian_augmentation_test_set = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="gaussian_augmentation",
    entry_point_kwargs=" ".join(
        [
            f"-P data_dir=/{DATASET_DIR}/training",
            "-P batch_size=20",
            "-P load_dataset_from_mlruns=true",
            f"-P dataset_run_id={response_deploy_le_net_patches_mnist_testing['mlflowRunId']}",
            "-P dataset_tar_name=adversarial_patch_dataset.tar.gz",
            "-P dataset_name=adv_patch_dataset",
            "-P gaussian_augmentation_sigma=0.3"
            
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_deploy_le_net_patches_mnist_testing["jobId"],
)


print("Gaussian Augmentation defense job submitted")
print("")
pprint.pprint(response_le_net_gaussian_augmentation_test_set)
print("")

response_le_net_gaussian_augmentation_test_set = get_run_id(response_le_net_gaussian_augmentation_test_set)

Gaussian Augmentation defense job submitted

{'createdOn': '2021-09-23T07:40:01.169829',
 'dependsOn': '297a740b-3289-4a5f-868a-4fcc95f62ff6',
 'entryPoint': 'gaussian_augmentation',
 'entryPointKwargs': '-P data_dir=//nfs/data/Mnist/training -P batch_size=20 '
                     '-P load_dataset_from_mlruns=true -P '
                     'dataset_run_id=26b2f57a935b418ba240508a6aecba0b -P '
                     'dataset_tar_name=adversarial_patch_dataset.tar.gz -P '
                     'dataset_name=adv_patch_dataset -P '
                     'gaussian_augmentation_sigma=0.3',
 'experimentId': 10,
 'jobId': '63dcf591-acd3-4c0b-88e1-ea8be7f34b82',
 'lastModified': '2021-09-23T07:40:01.169829',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/78144a90319f45e4a141fa73f3fa6b49/workflows.tar.gz'}



In [22]:
# Wait for defense to complete, then check baseline model on defended test set with adversarial patches.

response_evaluate_gaussian_augmentation_images = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_le_net_gaussian_augmentation_test_set['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_le_net",
            f"-P model_version=none",
            "-P batch_size=20",
            "-P adv_tar_name=gaussian_augmentation_dataset.tar.gz",
            "-P adv_data_dir=adv_testing",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_le_net_gaussian_augmentation_test_set["jobId"],
)

print("Inference job submitted")
print("")
pprint.pprint(response_evaluate_gaussian_augmentation_images)
print("")

response_evaluate_gaussian_augmentation_images = wait_until_finished(response_evaluate_gaussian_augmentation_images)
results = get_mlflow_results(response_evaluate_gaussian_augmentation_images)
print("Adversarial Patches with Gaussian Augmentation Results: \n")
pprint.pprint(results.data.metrics)


Inference job submitted

{'createdOn': '2021-09-23T07:40:08.362354',
 'dependsOn': '63dcf591-acd3-4c0b-88e1-ea8be7f34b82',
 'entryPoint': 'infer',
 'entryPointKwargs': '-P run_id=30e867af34c348bd80fd22cdb8f4de61 -P '
                     'model_name=howard_mnist_adversarial_patches_le_net -P '
                     'model_version=none -P batch_size=20 -P '
                     'adv_tar_name=gaussian_augmentation_dataset.tar.gz -P '
                     'adv_data_dir=adv_testing',
 'experimentId': 10,
 'jobId': '5c86e105-c96a-4fd2-af51-d71be70a3390',
 'lastModified': '2021-09-23T07:40:08.362354',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/1f9aea7668484d3681348959758d01ee/workflows.tar.gz'}

Adversarial Patches with Gaussian Augmentation Results: 

{'accuracy': 0.31963926553726196,
 'auc': 0.7049597501754761,
 'loss': 2.7053643875943876,
 'precision': 0.3955613672733307,
 'recall': 0.27324649691581726}


It appears that of the three preprocessing defenses, gaussian augmentation might actually improve patch attach effectiveness.

Since preprocessing defenses appear to be ineffective at stopping adversarial patches, let's try another approach.

In [23]:
%%bash

# Create the workflows.tar.gz file
make workflows

make: Nothing to be done for 'workflows'.


## Adversarial Training Defense:

Finally, we will train a new copy of the LeNet model on training set that contains adversarial patches.
In doing so, the model learns to ignore the adversarial patches.

In [24]:
# Finally, train and retest patched dataset.

response_deploy_le_net_patches_mnist_adv_training = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="train",
    entry_point_kwargs=" ".join(
        [
            f"-P dataset_run_id_testing={response_deploy_le_net_patches_mnist_testing['mlflowRunId']}",
            f"-P dataset_run_id_training={response_deploy_le_net_patches_mnist_training['mlflowRunId']}",
            "-P batch_size=256",
            "-P model_architecture=le_net",
            f"-P register_model_name={EXPERIMENT_NAME}_adversarial_patch_le_net",
            "-P epochs=30",
            f"-P data_dir_training={DATASET_DIR}/training",
            f"-P data_dir_testing={DATASET_DIR}/testing",
            "-P load_dataset_from_mlruns=True",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_deploy_le_net_patches_mnist_training["jobId"],
)

print("Patch adversarial training (LeNet-5 architecture) job submitted")
print("")
pprint.pprint(response_deploy_le_net_patches_mnist_adv_training)
print("")

response_deploy_le_net_patches_mnist_adv_training = get_run_id(response_deploy_le_net_patches_mnist_adv_training)

Patch adversarial training (LeNet-5 architecture) job submitted

{'createdOn': '2021-09-23T07:40:57.322801',
 'dependsOn': 'b7963146-e4b7-42a0-a37d-631336ed6dd8',
 'entryPoint': 'train',
 'entryPointKwargs': '-P '
                     'dataset_run_id_testing=26b2f57a935b418ba240508a6aecba0b '
                     '-P '
                     'dataset_run_id_training=81f11d97889e411d891727b1345aa58f '
                     '-P batch_size=256 -P model_architecture=le_net -P '
                     'register_model_name=howard_mnist_adversarial_patches_adversarial_patch_le_net '
                     '-P epochs=30 -P '
                     'data_dir_training=/nfs/data/Mnist/training -P '
                     'data_dir_testing=/nfs/data/Mnist/testing -P '
                     'load_dataset_from_mlruns=True',
 'experimentId': 10,
 'jobId': 'ac5c8c6a-9bde-4dd6-9182-b37623918bd9',
 'lastModified': '2021-09-23T07:40:57.322801',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '2

/home/hhuang/.conda/envs/tensorflow-mnist-classifier/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
response_evaluate_adv_training = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_deploy_le_net_patches_mnist_testing['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_adversarial_patch_le_net",
            f"-P model_version=none",
            "-P batch_size=20",
            "-P adv_tar_name=adversarial_patch_dataset.tar.gz",
            "-P adv_data_dir=adv_patch_dataset",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_deploy_le_net_patches_mnist_adv_training["jobId"],
)

print("Patch evaluation job submitted")
print("")
pprint.pprint(response_evaluate_adv_training)
print("")

response_evaluate_adv_training = wait_until_finished(response_evaluate_adv_training)
results = get_mlflow_results(response_evaluate_adv_training)
print("Adversarial Training Results:")

Patch evaluation job submitted

{'createdOn': '2021-09-23T07:41:05.517498',
 'dependsOn': 'ac5c8c6a-9bde-4dd6-9182-b37623918bd9',
 'entryPoint': 'infer',
 'entryPointKwargs': '-P run_id=26b2f57a935b418ba240508a6aecba0b -P '
                     'model_name=howard_mnist_adversarial_patches_adversarial_patch_le_net '
                     '-P model_version=none -P batch_size=20 -P '
                     'adv_tar_name=adversarial_patch_dataset.tar.gz -P '
                     'adv_data_dir=adv_patch_dataset',
 'experimentId': 10,
 'jobId': '77b59ddf-365d-42f6-af2c-7d19b81ff591',
 'lastModified': '2021-09-23T07:41:05.517498',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/bc0243c8a0124e24b6c5e22e129348a3/workflows.tar.gz'}

Adversarial Training Results:


**We can see that for adversarial patches, adversarial training appears to be a good option.**

**Some other interesting approaches that users may want to investigate by modfifying this demo are:**

-   Generating a separate set of patches for training and test data. 
    -   To do so, add a secondary patch generation job and link the run_id of this new job to one of the existing test/training patch deployment jobs.
-   Experimenting with alternate adversarially trained models. 
    -   If both the MNIST patch demo and MNIST FGM demos are run together, then users can swap the `model={EXPERIMENT_NAME}_adversarial_patch_le_net/None` with the model name of the FGM experiment.
    -   Doing so can let users quickly check which adversarial training offers robust protection against multiple types of attacks. 
     
**Please consult the README documentation for more information regarding available entrypoints and attack/defense parameters.**

<a id='querying_cell'></a>
## Querying the MLFlow Tracking Service

Currently the lab API can only be used to register experiments and start jobs, so if users wish to extract their results programmatically, they can use the `MlflowClient()` class from the `mlflow` Python package to connect and query their results.
Since we captured the run ids generated by MLFlow, we can easily retrieve the data logged about one of our jobs and inspect the results.
To start the client, we simply need to run,

In [26]:
mlflow_client = MlflowClient()

The client uses the environment variable `MLFLOW_TRACKING_URI` to figure out how to connect to the MLFlow Tracking Service, which we configured near the top of this notebook.
To query the results of one of our runs, we just need to pass the run id to the client's `get_run()` method.
As an example, let's query the run results for the patch attack applied to the LeNet-5 architecture,

In [27]:
run_le_net = mlflow_client.get_run(response_le_net_patches["mlflowRunId"])

If the request completed successfully, we should now be able to query data collected during the run.
For example, to review the collected metrics, we just use,

In [28]:
pprint.pprint(run_le_net.data.metrics)

{}


To review the run's parameters, we use,

In [29]:
pprint.pprint(run_le_net.data.params)

{'adv_data_dir': 'adv_patches',
 'adv_tar_name': 'adversarial_patch.tar.gz',
 'data_dir': '/nfs/data/Mnist/training',
 'dataset_seed': '959983847',
 'entry_point_seed': '147400987479783948928923446442624763887',
 'image_size': '28,28,1',
 'imagenet_preprocessing': 'false',
 'learning_rate': '5.0',
 'max_iter': '500',
 'model_name': 'howard_mnist_adversarial_patches_le_net',
 'model_version': 'none',
 'num_patch': '3',
 'num_patch_gen_samples': '40',
 'patch_target': '5',
 'rotation_max': '22.5',
 'scale_max': '1.0',
 'scale_min': '0.1',
 'seed': '-1',
 'tensorflow_global_seed': '803529856'}


To review the run's tags, we use,

In [30]:
pprint.pprint(run_le_net.data.tags)

{'mlflow.project.backend': 'securingai',
 'mlflow.project.entryPoint': 'gen_patch',
 'mlflow.source.name': '/work/tmpjemztbzq',
 'mlflow.source.type': 'PROJECT',
 'mlflow.user': 'securingai',
 'securingai.dependsOn': 'a52bb183-924f-491d-919e-7958e5371870',
 'securingai.jobId': '661f7ac9-251a-47dd-83ea-a54bd2cc196a',
 'securingai.queue': 'tensorflow_gpu'}


There are many things you can query using the MLFlow client.
[The MLFlow documentation gives a full overview of the methods that are available](https://www.mlflow.org/docs/latest/python_api/mlflow.tracking.html#mlflow.tracking.MlflowClient).